# Berechnung der Auswirkung der Meetinggröße
-> Keine statistisch signifikanten Wirkungen in unserer gewünschten Größenordnung

In [2]:
import numpy as np

### Cutler et al. 2021

##### Nachbauen der Ergebnisse
1. Nur statistisch signifikant

In [15]:
def logit(p):
    #log-odds function
    return np.log(p / (1 - p))

def inv_logit(log_odds):
    #Calculates the probability from log-odds
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM THE CAUSAL GRAPH (Figure 7) ---
COEFS_INCLUSIVE = {
    'Agenda': 1.4,
}
B0_INCLUSIVE = 0.0

COEFS_COMFORTABLE = {
    'Inclusive': 6.7, # Note: This is an intermediate variable input
}
B0_COMFORTABLE = 0.0

COEFS_EFFECTIVE = {
    'Agenda': 1.4,
}
B0_EFFECTIVE = 0.0



In [18]:

def calculate_meeting_effectiveness_probability(meeting_attributes: dict, B0_E=B0_EFFECTIVE):
    
    required_base_inputs = set([
        'Agenda', "Inclusive"
    ])
    
    missing_inputs = [key for key in required_base_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential input attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs.")
        return None
    
    #Calculate Log-Odds for Inclusive (Intermediate Variable) ---
    log_odds_inclusive = B0_INCLUSIVE
    for attr, coef in COEFS_INCLUSIVE.items():
        log_odds_inclusive += coef * meeting_attributes[attr]
            
    P_inclusive = inv_logit(log_odds_inclusive)
    
    #Convert P(Inclusive) to a predicted binary state (0 or 1) for the next stage
    Predicted_Inclusive = 1 if P_inclusive >= 0.5 else 0 
    
    #Calculate Log-Odds for Comfortable (Intermediate Variable) ---
    log_odds_comfortable = B0_COMFORTABLE
    
    log_odds_comfortable += COEFS_COMFORTABLE['Inclusive'] * Predicted_Inclusive 

    P_comfortable = inv_logit(log_odds_comfortable)
    Predicted_Comfortable = 1 if P_comfortable >= 0.5 else 0

    # --- 3. Calculate Log-Odds for Effective (Final Outcome) ---
    log_odds_effective = B0_E
    
    # Input 1: Inclusive (using the predicted binary state)
    log_odds_effective += COEFS_EFFECTIVE['Inclusive'] * Predicted_Inclusive
    
    # Input 2: Comfortable (using the predicted binary state)
    log_odds_effective += COEFS_EFFECTIVE['Comfortable'] * Predicted_Comfortable
    
    # Input 3: Agenda (base attribute)
    log_odds_effective += COEFS_EFFECTIVE['Agenda'] * meeting_attributes['Agenda']

    # --- 4. Calculate Final Probability of Effectiveness ---
    P_effective = inv_logit(log_odds_effective)
    
    print("--- Intermediate Results ---")
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print(f"Predicted Inclusive State: {Predicted_Inclusive}")
    print("-" * 28)
    print(f"Log-Odds Comfortable (BX_C): {log_odds_comfortable:.4f}")
    print(f"Predicted P(Comfortable): {P_comfortable:.4f}")
    print(f"Predicted Comfortable State: {Predicted_Comfortable}")
    print("-" * 28)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 28)
    
    return P_effective



In [17]:
## Scenario 1: An Ideal Meeting (High-Potential)
ideal_meeting_attributes = {
    "Agenda": 0, 'Inclusive':1
}

print("## Scenario 1: High-Potential Meeting (Ideal Attributes)")
probability_1 = calculate_meeting_effectiveness_probability(ideal_meeting_attributes)
print(f"Final Probability 1: {probability_1:.4f}")

print("\n" + "="*50 + "\n")

## Scenario 2: A Poor Remote Meeting (Low-Potential)
poor_remote_meeting_attributes = {
    "Agenda": 1, "Inclusive":1
}

print("## Scenario 2: Low-Potential Meeting (Poor Remote Attributes)")
probability_2 = calculate_meeting_effectiveness_probability(poor_remote_meeting_attributes)
print(f"Final Probability 2: {probability_2:.4f}")


## Scenario 1: High-Potential Meeting (Ideal Attributes)


KeyError: 'Inclusive'

2. Statistisch signifikant und nicht signifikant

In [12]:
def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM TABLE 5 (GLM Coefficients) ---
# NOTE: The table provides coefficients (beta_i) for each variable into four models (columns).
# Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.
# The variable 'JoinFromRemotely' is used in the table, not 'JoinRemotely'.

GLM_COEFFICIENTS = {
    # 1. Model for Participation (Input for Inclusive and Comfortable)
    # Participation = f(RemoteOnly, Comfortable, MeetingSize5p, JoinFromRemotely, Postmeet)
    'Participation': {
        'MeetingSize5p': -1.34,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 2. Model for Inclusive (Intermediate Variable)
    # Inclusive = f(RemoteOnly, Participation, Comfortable, MeetingSize5p, Video, Agenda, Quality, Reliability)
    'Inclusive': {
        'MeetingSize5p': -0.50,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 3. Model for Comfortable (Intermediate Variable)
    # Comfortable = f(RemoteOnly, Participation, MeetingSize5p, JoinFromRemotely, Agenda, Postmeet, Reliability)
    'Comfortable': {
        'MeetingSize5p': -0.13,
        'B0': 0.0 # Assumed Intercept
    },
    
    # 4. Model for Effective (Final Outcome)
    # Effective = f(RemoteOnly, Inclusive, Participation, Comfortable, MeetingSize5p, JoinFromRemotely, Video, Agenda, Premeet, Postmeet, Quality, Reliability)
    'Effective': {
        'MeetingSize5p': 0.01,
        'B0': 0.0 # Assumed Intercept
    }
}


def calculate_meeting_effectiveness_probability_v2(meeting_attributes: dict):
    """
    Calculates the probability of Effective using GLM coefficients from Table 5
    and cascading predicted probabilities (continuous values) for intermediate variables.
    """
    
    # --- 0. Define all required root inputs (from Table 5) ---
    required_root_inputs = set([
       'MeetingSize5p'
    ])
    
    # Check for missing required inputs
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in the table.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    # This must be calculated first as it feeds into Inclusive and Comfortable.

            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    # Depends on root inputs AND P_participation, P_comfortable (circular dependency, handle Comfortable later)
    # Note: In a proper GLM/PACM implementation, P_comfortable would be either predicted 
    # simultaneously or iteratively. For this linear chain, we'll calculate Inclusive and Comfortable 
    # based on the assumption that the *other* is a root input for now.
    
    # We will treat Comfortable's input on Inclusive as a zero for now, 
    # as the table shows a circular dependency (Inclusive->Comfortable and Comfortable->Inclusive).
    # Since the graph does not show a direct Comfortable->Inclusive edge, we will ignore the 1.89** for now.
    
    log_odds_inclusive = GLM_COEFFICIENTS['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]

    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Comfortable (P_comfortable) ---
    # Depends on root inputs AND P_participation
    
    log_odds_comfortable = GLM_COEFFICIENTS['Comfortable']['B0']
    for attr, coef in GLM_COEFFICIENTS['Comfortable'].items():
        if attr in meeting_attributes:
            log_odds_comfortable += coef * meeting_attributes[attr]

    P_comfortable = inv_logit(log_odds_comfortable)

    
    # --- 4. Calculate Effective (P_effective) ---
    # Depends on root inputs AND P_inclusive, P_participation, P_comfortable
    
    log_odds_effective = GLM_COEFFICIENTS['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive # Use continuous probability
        elif attr == 'Participation':
            log_odds_effective += coef * P_participation # Use continuous probability
        elif attr == 'Comfortable':
            log_odds_effective += coef * P_comfortable # Use continuous probability

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Table 5 Coefficients) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 45)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 45)
    print(f"Log-Odds Comfortable (BX_C): {log_odds_comfortable:.4f}")
    print(f"Predicted P(Comfortable): {P_comfortable:.4f}")
    print("-" * 45)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 45)
    
    return P_effective


# --- RE-DEFINING INPUT SCENARIOS FOR TABLE 5 ---

## SCENARIO 1: Ideal Meeting (Maximum Positive Inputs)
# Note: JoinFromRemotely is the new variable from the table, replacing JoinRemotely.
ideal_meeting_attributes_v2 = {
    'Agenda': 1, # Agenda used
}

print("## Scenario 1: High-Potential Meeting (Using Table 5 Coefficients)")
probability_1_v2 = calculate_meeting_effectiveness_probability_v2(ideal_meeting_attributes_v2)
print(f"Final Probability 1: {probability_1_v2:.4f}")

print("\n" + "="*70 + "\n")

## SCENARIO 2: Poor Remote Meeting (Maximum Negative/Minimizing Positive Inputs)
poor_remote_meeting_attributes_v2 = {

    'Agenda': 0, # No agenda
   
}

print("## Scenario 2: Low-Potential Meeting (Using Table 5 Coefficients)")
probability_2_v2 = calculate_meeting_effectiveness_probability_v2(poor_remote_meeting_attributes_v2)
print(f"Final Probability 2: {probability_2_v2:.4f}")

## Scenario 1: High-Potential Meeting (Using Table 5 Coefficients)
ERROR: Missing essential root attributes: MeetingSize5p
Please provide values (0 or 1) for all root inputs defined in the table.


TypeError: unsupported format string passed to NoneType.__format__

### Hosseinkashi et al. 2024
##### Nachbauen der Ergebnisse
1. statistich signifikant

In [2]:
import numpy as np
import math # Import the math module for ln

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM TABLE 3 (Combined Model, only statistically significant ORs) ---
# Odds Ratios (OR) were converted to GLM Coefficients (beta_i = ln(OR)).
# Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.

GLM_COEFFICIENTS_V3 = {
    'Participation': {
        'Small Meeting (8 Or Less)': math.log(7.13), # 1.964 
        'B0': 0.0
    },
    
    'Inclusive': {
        'Small Meeting (8 Or Less)': math.log(1.51), # 0.412
        'B0': 0.0
    },
    
    'Effective': {
        'Small Meeting (8 Or Less)': math.log(1.29), # 0.255
        'B0': 0.0
    }
}


def calculate_meeting_effectiveness_probability_v3(meeting_attributes: dict):

    # --- 0. Define all required root inputs ---
    required_root_inputs = set(
        list(GLM_COEFFICIENTS_V3['Participation'].keys()) +
        list(GLM_COEFFICIENTS_V3['Inclusive'].keys()) +
        list(GLM_COEFFICIENTS_V3['Effective'].keys())
    )
    # Entferne die berechneten Variablen und Intercepts
    required_root_inputs.difference_update(['B0', 'Participation', 'Inclusive']) 
    
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in the model.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    log_odds_part = GLM_COEFFICIENTS_V3['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    log_odds_inclusive = GLM_COEFFICIENTS_V3['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation 
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Effective (P_effective) ---
    log_odds_effective = GLM_COEFFICIENTS_V3['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive 

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 65)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 65)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 65)
    
    return P_effective


## SCENARIO 1: Ideal Meeting (Optimal)
ideal_meeting_attributes_v3 = {
    'Small Meeting (8 Or Less)': 1, 
}

print("## Szenario 1: Optimales Meeting (Tabelle 3)")
probability_1_v3 = calculate_meeting_effectiveness_probability_v3(ideal_meeting_attributes_v3)
print(f"Final Probability 1: {probability_1_v3:.4f}")

print("\n" + "="*75 + "\n")

## SCENARIO 2: Poor Meeting (Schlecht)
poor_meeting_attributes_v3 = {
   
    'Small Meeting (8 Or Less)': 0
}

print("## Szenario 2: Schlechtes Meeting (Tabelle 3)")
probability_2_v3 = calculate_meeting_effectiveness_probability_v3(poor_meeting_attributes_v3)
print(f"Final Probability 2: {probability_2_v3:.4f}")

## Szenario 1: Optimales Meeting (Tabelle 3)
--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---
Log-Odds Participation (BX_P): 1.9643
Predicted P(Participation): 0.8770
-----------------------------------------------------------------
Log-Odds Inclusive (BX_I): 0.4121
Predicted P(Inclusive): 0.6016
-----------------------------------------------------------------
Log-Odds Effective (BX_E): 0.2546
Final Predicted P(Effective): 0.5633
-----------------------------------------------------------------
Final Probability 1: 0.5633


## Szenario 2: Schlechtes Meeting (Tabelle 3)
--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---
Log-Odds Participation (BX_P): 0.0000
Predicted P(Participation): 0.5000
-----------------------------------------------------------------
Log-Odds Inclusive (BX_I): 0.0000
Predicted P(Inclusive): 0.5000
-----------------------------------------------------------------
Log-Odds Effective (BX_E): 0.0000
Final Predicted P(Effect